# basic population plots for fly sleep behaviour based on rethomics pipeline

# 1. load packages

In [ ]:
devtools::install_github("shaliulab/behavr@deployment")
devtools::install_github("shaliulab/scopr@deployment")
devtools::install_github("shaliulab/sleepr@deployment")
devtools::install_github("shaliulab/ggetho@deployment")

library(devtools)
install_github("rethomics/ggetho")

library(behavr)
library(scopr)
library(sleepr)
library(ggplot2)
library(ggetho)
library(tidyverse)

# 2. load and link metadata to results folder

In [ ]:
metadata_10x <- fread("/Users/Joana/sleep_signature/behaviour/10x/metadata.csv")
metadata_10x$date <- as.character(metadata_10x$date)
class(metadata_10x$date)

metadata_10x <- link_ethoscope_metadata(metadata_10x,
                                              result_dir = "/2021-04-14_ethoscope_data/results")
                                              
system.time(
  dt_10x <- load_ethoscope(metadata_10x,
                                 reference_hour=NA,
                                 FUN = sleepr::sleep_annotation,
                                 cache = "/ethoscope_data/cache",
                                 verbose=TRUE,
                                 map_arg=list(velocity_correction_coef="velocity_correction_coef")
  )
)

# 3. add phase column (light or dark phase) based on time

In [ ]:
dt_10x[, phase := ifelse(t %% hours(24) < hours(12), "L", "D")]

# 4. prepare data before plotting

In [ ]:
metadata_10x$ZT_Time <- factor(metadata_10x$ZT_Time,
                                 levels = c("ZT 2", "ZT 8", "ZT 14", "ZT 20"),ordered = TRUE)

metadata_10x$condition <- factor(metadata_10x$condition,
                                 levels = c("2h rebound sleep (RS ZT0-2)",
                                            "14h sleep deprivation (SD ZT12-2)",
                                            "20h GBX-sleep (GBX-S ZT12-8)", 
                                            "8h wake (W ZT0-8)", 
                                            "8h sleep deprivation (SD ZT0-8)",
                                            "20h sleep deprivation (SD ZT12-8)",
                                            "2h sleep (S ZT12-14)", 
                                            "2h sleep deprivation (SD ZT12-14)",
                                            "8h sleep (S ZT12-20)", 
                                            "8h yoked control (YC ZT12-20)", 
                                            "8h sleep deprivation (SD ZT12-20)")
                                 ,ordered = TRUE)
setkey(metadata_10x, id)
setkey(dt_10x, id)
setmeta(dt_10x, metadata_10x)


# 4. plot population sleep plot

In [ ]:
ggetho(dt_10x, aes(y=asleep, colour=treatment, fill=treatment)) +
  ggetho::stat_pop_etho() +
  stat_ld_annotations() +
  facet_grid(condition ~ .) +
  theme_classic(base_size=30)+
  scale_y_continuous("time asleep", breaks=c(0, 0.5, 1), labels = scales::percent)+
  theme(axis.title.x=element_blank(),
        axis.ticks.x=element_blank(), 
        axis.text.x=element_blank(), 
        axis.text.y=element_text(size=10, colour = "black"),
        axis.title.y=element_text(size=40, colour = "black")) +
  scale_colour_manual(values = c("#1f77b4", "#d62728"))+
  scale_fill_manual(values = c("#1f77b4", "#d62728"))+
  theme(strip.text.y.right = element_text(angle = 0))

ggsave("/Users/Joana/14122022_10x_conditions_combined.pdf", width = 55, height = 20, dpi = 300, units = "cm")